<a href="https://colab.research.google.com/github/Sabbam/Ai-Travel_planner/blob/main/Model_AIPFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json


In [181]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import firebase_admin
from firebase_admin import credentials, db

# Initialize Firebase Admin SDK
try:
    firebase_admin.get_app()
except ValueError:
    cred = credentials.Certificate('/content/ai-manager-1086e-firebase-adminsdk-fbsvc-fee8cb3cb9.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://your-firebase-database-url.firebaseio.com/'
    })

# Fetch data from Firebase
ref = db.reference("/")
data = ref.get()

# Function to get motivational quotes based on savings grade
def get_motivational_quote(grade):
    quotes = {
        "Excellent": "You're doing amazing! Keep up the great savings habit! 💰",
        "Good": "Good job! A little more effort and you’ll reach financial freedom! 🚀",
        "Average": "You're on the right track. Try cutting unnecessary expenses! ✂️",
        "Poor": "Don't worry! Every small step counts. Start today, and you’ll improve! 🌱"
    }
    return quotes.get(grade, "Keep pushing forward towards your financial goals!")

# Function to provide personalized spending advice
def get_spending_advice(row):
    advice = []

    if row['food'] > 0.3 * row['total_income'] and row['medicines'] > 0.2 * row['total_income']:
        advice.append("Reduce excessive food expenses to maintain better health! 🍎")

    if row['entertainment'] > 0.3 * row['total_income'] and row['savings_ratio'] < 0.1:
        advice.append("Consider reducing entertainment expenses to improve savings! 🎟️")

    if row['transportation'] > 0.3 * row['total_income']:
        advice.append("Try carpooling or public transport to cut transportation costs! 🚗")

    if row['clothing'] > 0.2 * row['total_income']:
        advice.append("Reevaluate clothing purchases to focus on essentials! 👕")

    return " | ".join(advice) if advice else "You're managing expenses well! 👍"

# Extract user expenses including correct income fields
def extract_expense_data(data):
    users = data.get("users", {})
    records = []

    for user_id, user_info in users.items():
        email = user_info.get("email", "N/A")
        income_data = user_info.get("income", {})
        latest_total_income = max((int(entry.get("totalIncome", 0) or 0) for entry in income_data.values()), default=0)
        expenses = user_info.get("expenses", {})

        for expense in expenses.values():
            if not isinstance(expense, dict):
                continue

            record = {
                "user_id": user_id,
                "email": email,
                "date": pd.to_datetime(expense.get("date", "1970-01-01")),
                "food": int(expense.get("food", "0") or "0"),
                "entertainment": int(expense.get("entertainment", "0") or "0"),
                "transportation": int(expense.get("transportation", "0") or "0"),
                "medicines": int(expense.get("medicines", "0") or "0"),
                "clothing": int(expense.get("clothing", "0") or "0"),
                "total_amount": int(expense.get("totalAmount", "0") or "0"),
                "total_income": latest_total_income,
            }
            records.append(record)

    return pd.DataFrame(records)

df = extract_expense_data(data)

# Aggregate expenses per category by month
df['month'] = df['date'].dt.to_period('M')
monthly_expense = df.groupby(['user_id', 'month']).agg({
    'food': 'sum',
    'entertainment': 'sum',
    'transportation': 'sum',
    'medicines': 'sum',
    'clothing': 'sum',
    'total_amount': 'sum',
    'total_income': 'sum'
}).reset_index()

# Calculate Savings Ratio & Financial Grade
monthly_expense['savings'] = monthly_expense['total_income'] - monthly_expense['total_amount']
monthly_expense['savings_ratio'] = monthly_expense['savings'] / monthly_expense['total_income']

def assign_financial_grade(savings_ratio):
    if savings_ratio > 0.5:
        return "Excellent"
    elif savings_ratio > 0.2:
        return "Good"
    elif savings_ratio > 0:
        return "Average"
    else:
        return "Poor"

monthly_expense['financial_grade'] = monthly_expense['savings_ratio'].apply(assign_financial_grade)
monthly_expense['motivational_quote'] = monthly_expense['financial_grade'].apply(get_motivational_quote)
monthly_expense['spending_advice'] = monthly_expense.apply(get_spending_advice, axis=1)

# Prepare dataset for ML model
features = ['food', 'entertainment', 'transportation', 'medicines', 'clothing', 'total_income']
target = 'savings_ratio'

X = monthly_expense[features]
y = monthly_expense[target]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "/content/scaler.pkl")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build and train the ML model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Save the trained model
model.save("/content/spending_behavior_model.keras")

print("✔ Model training complete & saved!")
print("✔ Motivational quotes and spending advice integrated!")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.8246 - mae: 0.9076 - val_loss: inf - val_mae: inf
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.7309 - mae: 0.8535 - val_loss: inf - val_mae: inf
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.6449 - mae: 0.7989 - val_loss: inf - val_mae: inf
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.5691 - mae: 0.7455 - val_loss: inf - val_mae: inf
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.5041 - mae: 0.6945 - val_loss: inf - val_mae: inf
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.4463 - mae: 0.6433 - val_loss: inf - val_mae: inf
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.3965 - mae: 0.5943 - val_loss: inf - val_mae: inf
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.3530 - mae: 0.5462 - val_loss: inf - val_mae: inf
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.3157 - mae: 0.4988 - val_loss: inf - val_mae: inf
Epoch 10/50
1/1 ━━━━━━━━━━━━

In [182]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import requests
from sklearn.preprocessing import StandardScaler
from fpdf import FPDF
from datetime import datetime

# Firebase Database URL
FIREBASE_URL = "https://ai-manager-1086e-default-rtdb.firebaseio.com/users.json"

# Load model & scaler
model = keras.models.load_model("/content/spending_behavior_model.keras", compile=False)
scaler = joblib.load("/content/scaler.pkl")

# Fetch real-time data from Firebase
def fetch_firebase_data():
    response = requests.get(FIREBASE_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Error fetching data from Firebase")
        return {}
# Extract expense data and goal data
def extract_expense_data(data, user_id):
    user_info = data.get(user_id, {})
    expenses = user_info.get("expenses", {})
    income_data = user_info.get("income", {})
    goals_data = user_info.get("goals", {})  # Fetching goals data

    # Extract the most recent goal amount (assuming the most recent goal is the one we need)
    latest_goal_amount = 0
    if goals_data:
        latest_goal_entry = max(goals_data.values(), key=lambda x: datetime.strptime(x["date"], "%m/%d/%Y, %I:%M:%S %p"))
        latest_goal_amount = int(latest_goal_entry.get("goalAmount", "0"))

    latest_income = max((int(entry.get("totalIncome", "0") or "0") for entry in income_data.values()), default=0)

    records = []
    for expense in expenses.values():
        date = pd.to_datetime(expense.get("date", "1970-01-01"))
        other_categories = {cat.get("name", "Other").strip(): int(cat.get("amount", "0")) for cat in expense.get("otherCategories", [])}

        record = {
            "date": date,
            "food": int(expense.get("food", "0") or "0"),
            "entertainment": int(expense.get("entertainment", "0") or "0"),
            "transportation": int(expense.get("transportation", "0") or "0"),
            "medicines": int(expense.get("medicines", "0") or "0"),
            "clothing": int(expense.get("clothing", "0") or "0"),
            "total_amount": int(expense.get("totalAmount", "0") or "0"),
            "total_income": latest_income,
            "goal_amount": latest_goal_amount,  # Including the latest goal amount
            "other_categories": other_categories
        }
        records.append(record)
    return pd.DataFrame(records)

# Generate bar chart for spending breakdown
def generate_bar_chart(user_id, category_data):
    categories, values = zip(*category_data.items())
    plt.figure(figsize=(12, 7))
    sns.barplot(x=list(categories), y=list(values), palette="coolwarm", edgecolor="black")
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel("Categories", fontsize=14)
    plt.ylabel("Amount Spent (INR)", fontsize=14)
    plt.title(f'Spending Breakdown for {user_id}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    chart_path = f"/content/{user_id}_bar_chart.png"
    plt.savefig(chart_path, bbox_inches='tight')
    plt.close()
    return chart_path
# Provide spending insights based on scenarios
def spending_analysis(category_data, other_categories=[]):
    insights = []
    total_spending = sum(category_data.values()) + sum(item['amount'] for item in other_categories)

    if total_spending == 0:
        return "No spending data available."

    for category, amount in category_data.items():
        percentage = (amount / total_spending) * 100

        if percentage > 40:
            insights.append(f"Significant spending detected in {category}. Consider optimizing expenses and prioritizing essential needs.")
        elif percentage > 30:
            insights.append(f"Moderate expenditure in {category}. Keeping a close watch will help maintain financial stability.")
        elif percentage < 10:
            if category == "transportation":
                min_target = 0.1 * total_spending
                max_target = 0.15 * total_spending
                insights.append(f"Limited spending on transportation. Efficient planning could set an ideal range between INR {min_target:.2f} and INR {max_target:.2f}.")
            elif category == "entertainment":
                insights.append(f"Minimal spending in entertainment. If this reflects your lifestyle, it's commendable, but ensure a balanced approach to leisure.")
            elif category == "medicines":
                insights.append(f"Low expenditure on medicines. If health needs are met, it's fine, but always ensure essential medical provisions.")
            elif category == "clothing":
                insights.append(f"Minimal spending on clothing. If intentional, it's efficient, but consider periodic updates for necessities.")
            else:
                insights.append(f"Low allocation for {category}. Re-evaluating based on necessity and priority could be beneficial.")

    for item in other_categories:
        name = item['name'].strip()
        amount = float(item['amount'])
        percentage = (amount / total_spending) * 100

        if percentage > 40:
            insights.append(f"High expenditure on {name}. Consider managing costs to ensure balanced spending.")
        elif percentage > 30:
            insights.append(f"Moderate spending on {name}. Keep monitoring to maintain financial discipline.")
        elif percentage < 10:
            insights.append(f"Minimal spending on {name}. This could be intentional, but ensure priorities are well-managed.")

    if not insights:
        insights.append("Your spending distribution is well-balanced. Excellent financial planning!")

    return "\n".join(insights)



# Generate PDF report
def generate_pdf_report(user_id, total_income, total_spending, predicted_savings, savings_grade, bar_chart_path, category_data, spending_advice, goal_amount, prediction_basis, spending_grade, spending_ratio):
    pdf = FPDF()
    pdf.add_page()

    pdf.set_font("Arial", style='B', size=18)
    pdf.cell(200, 10, txt="AI Powered Finance Manager", ln=True, align='C')
    pdf.ln(5)

    pdf.set_font("Arial", size=14)
    pdf.cell(200, 10, txt=f"Spending Report for {user_id}", ln=True, align='C')
    pdf.ln(10)

    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Total Income: INR {total_income}", ln=True)
    pdf.cell(200, 10, txt=f"Total Spending (Last 30 Days): INR {total_spending}", ln=True)
    pdf.cell(200, 10, txt=f"Goal Amount: INR {goal_amount}", ln=True)
    pdf.cell(200, 10, txt=f"Predicted Savings: INR {predicted_savings:.2f}", ln=True)
    pdf.cell(200, 10, txt=f"Savings Grade: {savings_grade}", ln=True)
    pdf.cell(200, 10, txt=f"Spending Ratio: {spending_ratio:.2f}", ln=True)
    pdf.cell(200, 10, txt=f"Spending Grade: {spending_grade}", ln=True)
    pdf.ln(10)

    pdf.set_font("Arial", style='B', size=10)
    pdf.cell(40, 10, "Category", border=1, align='C')
    pdf.cell(40, 10, "Amount (INR)", border=1, align='C')
    pdf.ln()

    pdf.set_font("Arial", size=10)
    for category, amount in category_data.items():
        pdf.cell(40, 10, category, border=1, align='C')
        pdf.cell(40, 10, f"INR {amount}", border=1, align='C')
        pdf.ln()

    pdf.ln(10)
    pdf.image(bar_chart_path, x=20, y=None, w=170)
    pdf.ln(10)

    # Spending Analytics Box
    pdf.set_font("Arial", style='B', size=12)
    pdf.set_text_color(255, 0, 0)
    pdf.cell(0, 10, "Spending Analytics:", border=1, ln=True, align='C')
    pdf.set_text_color(0, 0, 0)
    pdf.multi_cell(0, 10, spending_advice.encode('latin-1', 'ignore').decode('latin-1'), border=1)
    pdf.ln(5)

    # Prediction Basis Box
    pdf.set_font("Arial", style='B', size=12)
    pdf.set_text_color(255, 0, 0)
    pdf.cell(0, 10, "Prediction Basis:", border=1, ln=True, align='C')
    pdf.set_text_color(0, 0, 0)

    prediction_basis = (
    "The predicted savings are based on the relationship between your current spending patterns "
    "and your income. The model analyzes your spending across categories like food, entertainment, "
    "transportation, medicines, and clothing to predict the portion of your income that can be saved."
    )

    pdf.multi_cell(0, 10, prediction_basis, border=1)
    pdf.ln(10)


    # Digitally Signed Note
    pdf.set_font("Arial", style='B', size=12)
    pdf.cell(0, 10, "AI Powered Finance Manager Development Team", ln=True, align='C')
    pdf.ln(10)

    output_path = f"/content/{user_id}_report.pdf"
    pdf.output(output_path, "F")
    print(f"✔ Report saved: {output_path}")

# Run for a test user
user_id_to_test = "aiicT0lnFvMG9NhUX066WHsRjGz1"
predict_savings(user_id_to_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
<ipython-input-182-5cf2b7e85d8f>:67: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=list(categories), y=list(values), palette="coolwarm", edgecolor="black")


🔍 Spending Analysis for aiicT0lnFvMG9NhUX066WHsRjGz1:
Moderate expenditure in food. Keeping a close watch will help maintain financial stability.
Limited spending on transportation. Efficient planning could set an ideal range between INR 7422.80 and INR 11134.20.
✔ Report saved: /content/aiicT0lnFvMG9NhUX066WHsRjGz1_report.pdf
